In [15]:
import pickle
import pandas as pd

In [31]:
year = 2021
month = 2
input_file= f'../01-into/data/fhv_tripdata_{year:04d}-{month:02d}.parquet'
output_file= f'./data/fhv_tripdata_{year:04d}-{month:02d}_pred.parquet'

In [17]:
with open('./models/model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

/Users/LabidiD/.local/share/virtualenvs/mlops-bootcamp-doULHIbJ/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/LabidiD/.local/share/virtualenvs/mlops-bootcamp-doULHIbJ/lib/python3.10/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [18]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [19]:
# broken url
# df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_????-??.parquet')
df = read_data(input_file)

In [20]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [21]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [22]:
y_pred.mean()

16.191691679979066

In [14]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

NameError: name 'year' is not defined

In [26]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_result'] = y_pred

In [27]:
df_result.head()

,ride_id,predicted_result
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206


In [32]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [33]:
!ls -lh ./data

total 38504
-rw-r--r--  1 LabidiD  staff    19M Jul 15 00:12 fhv_tripdata_2021-02_pred.parquet
